In [1]:
#!/usr/bin/env python3
from traitlets.config.manager import BaseJSONConfigManager
cm = BaseJSONConfigManager()
cm.update(
    "rise",
    {
        "transition": "zoom",
        "start_slideshow_at": "selected",
        "footer": "<h3>Python Girona 2018</h3>"
    }
)

{'transition': 'zoom',
 'start_slideshow_at': 'selected',
 'footer': '<h3>Python Girona 2018</h3>'}

# Docker 101

## What is a Container?

A `container` embeds an application / service and all of its dependencies in a single space.

- Isolation

- Ready to run

- Standardization //run everywhere

## What is Docker?

`Docker` is a platform for developers and sysadmins to develop, ship, and run applications

- `Docker Engine`: open source containerization technology

- `Docker Hub`: SaaS service for sharing and managing app stacks

## Docker vs VM?

![alt text](ct_vs_vm.png "Containers vs VMs")



images from https://www.docker.com/resources/what-container


## Benefits

- Smaller costs (by default)

- Fast delivery
  - Compatibility
  - Maintainability
  - CI

- Rapid deployment

- Scalability

- Security

## How to use it?

### Dockerfile

- defines the behaviour of the image

- is like a Makefile that prepares everything

- also starts the related service

```
FROM alpine:latest
COPY . /app
RUN make /app
CMD python /app/app.py
```

### Docker Image

- file

- comprised of multiple layers  //~ snapshot incremental diffs 

- used to execute code in a `Docker container`

//exemple a depurar només com a idea
```
$ docker images
REPOSITORY                        TAG                 IMAGE ID            CREATED             SIZE
shodand_scanner_scanner           latest              82daf18d5d92        11 days ago         551MB
redis                             alpine              05097a3a0549        12 days ago         30MB
elpaso/qgis-testing-environment   master              334775a61a4f        2 weeks ago         3.39GB
docker_erp                        latest              285af92a3352        4 weeks ago         1.05GB
ubuntu                            16.04               b9e15a5d1e1a        5 weeks ago         115MB
python                            2.7                 4ee4ea2f0113        5 weeks ago         908MB
elpaso/qgis-testing-environment   <none>              2690c949040f        7 weeks ago         3.31GB
empireproject/empire              latest              527d5d78e7fc        3 months ago        1.19GB
mongo                             3.0                 fdab8031e252        5 months ago        232MB
gisce/oopostgres_test             postgis             6951e682bee1        2 years ago         790MB
redis                             2.8                 481995377a04        2 years ago         186MB
```